## In order to use this script , we you need to instal the following dependencies 

#### -> uv pip install selenium beautifulsoup4 google-generativeai webdriver-manager


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from openai import OpenAI
import os
from bs4 import BeautifulSoup
import time
import google.generativeai as genai

#### 1.selenium scrapping 

In [4]:
# selenium setup 
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)


In [28]:
def scrape_page ( url ):

    driver.get(url)
    time.sleep(3)  #allowing the JS to render

    soup =BeautifulSoup(driver.page_source, "html.parser")

    # removing junk / unecessary tags
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose();
    
    text =  soup.get_text(separator=" ")

    return " ".join(text.split()) 

In [30]:
def trim_text (text, max_chars=12000):
    return text[:max_chars]

#### 2.calling gemini 

In [35]:
import os
from openai import OpenAI


GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"


google_api_key = os.getenv("GEMINI_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [40]:
gemini =  OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=google_api_key)

system_prompt = """ you are a professional content summarizer,
Your task is to generate a clear ,concise and structured summary. 
Avoid fluff. Focus on key ideas / highlights + making them understandable 

output should have bullet points and proper spacing , to maintain readability"""

def user_prompt(page_text):
    return f"""
    Summarize the following webpage content in bullet points:

    {page_text}
"""


In [54]:
def gemini_summarize(text):

    response = gemini.chat.completions.create(
        model="gemini-2.5-pro",
         messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(text)}
    ])

    return  response.choices[0].message.content
    

In [ ]:
url = "https://en.wikipedia.org/wiki/Web_scraping"

text = scrape_page(url)
# print(text)
trimmed_text = trim_text(text)

 #limited to 12000 words
# print(trimmed_text) 

print(gemini_summarize(trimmed_text))